In [1]:
using Dates
using ArgParse
using Printf
using Random

include("../btg.jl")
include("../datasets/load_abalone.jl");

In [2]:
# shuffle data
ind_shuffle = randperm(MersenneTwister(1234), size(data, 1)) 
data = data[ind_shuffle, :]
target = target[ind_shuffle]
id_train = 1:400; posx = 1:7; posc = 1:3
x = data[id_train, posx] 
Fx = data[id_train, posc] 
y = float(target[id_train])
trainingData1 = trainingData(x, Fx, y) #training data used for testing various functions
d = getDimension(trainingData1); n = getNumPts(trainingData1); p = getCovDimension(trainingData1)

3

In [3]:
#parameter setting
rangeθ = [10.0 1000]
rangeθm = repeat(rangeθ, d, 1)
rangeλ = [-1. 1.] #we will always used 1 range scale for lambda
myquadtype = ["SparseGrid", "Gaussian"]
myquadtype = ["QuasiMonteCarlo", "QuasiMonteCarlo"]
btg1 = btg(trainingData1, rangeθm, rangeλ; quadtype = myquadtype)
(pdf, cdf, dpdf, quantInfo) = solve(btg1); 

In [4]:
id_test = 1001:1200
n_test = length(id_test)
id_fail = []
Fx0 = reshape(data[id_test, posc], n_test, length(posc))
x0 = reshape(data[id_test, posx], n_test, length(posx)) 
count_test = 0
error_abs  = 0.
error_sq   = 0.
nlpd       = 0.

0.0

In [5]:
i = 1
x0_i = reshape(x0[i, :], 1, length(posx))
Fx0_i = reshape(Fx0[i, :], 1, length(posc))
y_i_true = getLabel(btg1.trainingData)[i]

0.27586206896551724

In [6]:
quantbound_fixed(p) = quantInfo(x0_i, Fx0_i, p) 
pdf_fixed(y) = pdf(x0_i, Fx0_i, y)
cdf_fixed(y) = cdf(x0_i, Fx0_i, y)
dpdf_fixed(y) = dpdf(x0_i, Fx0_i, y)

dpdf_fixed (generic function with 1 method)

In [10]:
support = [.01, 5.]support = [.01, 5.]
function support_comp!(pdf, support)
      current = pdf(support[1])
      while i in 1:5
        next = pdf(support[1]/5)
        if next < current
          support[1] /= 5
        else
          break
        end
        current = next
      end
      while pdf(support[2]) > 1e-6 
        support[2] *= 1.2
      end
      # should make sure CDF(support[2]) - CDF(support[1]) > .97 to make 95% CI possible
      INT = cdf_fixed(support[2]) - cdf_fixed(support[1])
      @assert INT > .97 "pdf integral $INT"
      return nothing
end
support_comp!(pdf_fixed, support)

ErrorException: syntax: "[0.01, 5]" is not a valid function argument name

In [8]:
support = [.01, 5.]

2-element Array{Float64,1}:
 0.01
 5.0

In [42]:
function test_single(i)
    x0_i = reshape(x0[i, :], 1, length(posx))
    Fx0_i = reshape(Fx0[i, :], 1, length(posc))
    pdf_i, cdf_i, dpdf_i, quantbound_i, support_i, fail_i = pre_process(x0_i, Fx0_i, pdf, cdf, dpdf, quantInfo)
    y_i_true = getLabel(btg1.trainingData)[i]
    median_i = quantile(cdf_i, quantbound_i, support_i)[1]
    try 
        CI_i = credible_interval(cdf_i, quantbound_i, support_i; mode=:equal, wp=.95)[1]
        count_test += (y_i_true >= CI_i[1])&&(y_i_true <= CI_i[2]) ? 1 : 0
    catch err
        append!(id_fail, i)
    end
end

test_single (generic function with 1 method)

In [ ]:
@time test_single(1)

In [23]:
pdf_i, cdf_i, dpdf_i, quantbound_i, support_i, fail_i = pre_process(x0_i, Fx0_i, pdf, cdf, dpdf, quantInfo)

BoundsError: SYSTEM: show(lasterr) caused an error

In [ ]:
median_i = quantile(cdf_i, quantbound_i, support_i)[1]

In [ ]:
CI_i = credible_interval(cdf_i, quantbound_i, support_i; mode=:equal, wp=.95)[1]

In [20]:
for i in 1:n_test
    @info "Current iteration", i
    x0_i = reshape(x0[i, :], 1, length(posx))
    Fx0_i = reshape(Fx0[i, :], 1, length(posc))
    pdf_i, cdf_i, dpdf_i, quantbound_i, support_i, fail_i = pre_process(x0_i, Fx0_i, pdf, cdf, dpdf, quantInfo)
    y_i_true = getLabel(btg1.trainingData)[i]
    median_i = quantile(cdf_i, quantbound_i, support_i)[1]
    try 
        CI_i = credible_interval(cdf_i, quantbound_i, support_i; mode=:equal, wp=.95)[1]
        count_test += (y_i_true >= CI_i[1])&&(y_i_true <= CI_i[2]) ? 1 : 0
    catch err
        append!(id_fail, i)
    end
    error_abs += abs(y_i_true - median_i)
    error_sq += (y_i_true - median_i)^2
    nlpd += log(pdf_i(y_i_true)) 
end

┌ Info: ("Current iteration", 1)
└ @ Main In[20]:2
┌ Info: ("Current iteration", 2)
└ @ Main In[20]:2
┌ Info: ("Current iteration", 3)
└ @ Main In[20]:2


InterruptException: InterruptException:

In [11]:
count_test /= n_test - length(id_fail)
error_abs  /= n_test
error_sq   /= n_test
nlpd       /= -n_test

-0.0

In [ ]:
if filesize("Exp_abalone_test.txt") == 0
    # write the setting headers
    io1 = open("Exp_abalone_test.txt", "w") 
    write(io1, "Time     ;    ind_train  ;  ind_test   ;      quadtype     ;     rangeθ    ;    rangeλ   ;  error_abs ;  error_sq ;  CI accuracy ; -log(p(y));  failed id \n")
    close(io1)
end

io1 = open("Exp_abalone_test.txt", "a") 
write(io1, "$(Dates.now())  ;    $ind    ;   $id_test   ;     $myquadtype     ;   $rangeθ   ;  $rangeλ ;   $count_test    ;  $error_abs   ;  $error_sq   ;   $nlpd   ;  $id_fail \n")
close(io1)


In [25]:
id_fail

3-element Array{Any,1}:
 1
 1
 1

In [12]:
a = []

0-element Array{Any,1}

In [15]:
append!(a, 1)

2-element Array{Any,1}:
 1
 1

In [16]:
length(a)

2